In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG

import pandas_datareader.data as web
import requests_cache
import datetime

In [2]:
expire_after = datetime.timedelta(days=30)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)

In [3]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2019, 9, 27)

moex_df = web.DataReader(['SIBN', 'GAZP', 'LSRG'], 'moex', start=start, end=end, session=session)

In [4]:
moex_df = moex_df.rename(columns={'VOLUME': 'Volume', 'OPEN': 'Open', 'HIGH': 'High', 'LOW': 'Low', 'CLOSE': 'Close', 'VALUE': 'Value'})

In [5]:
sibn_df = moex_df[(moex_df['BOARDID'] == 'TQBR') & (moex_df['SECID'] == 'SIBN')][['SHORTNAME', 'SECID', 'BOARDID', 'NUMTRADES', 'Value', 'Volume', 'Open', 'High', 'Low', 'Close']]

In [6]:
gazp_df = moex_df[(moex_df['BOARDID'] == 'TQBR') & (moex_df['SECID'] == 'GAZP')][['SHORTNAME', 'SECID', 'BOARDID', 'NUMTRADES', 'Value', 'Volume', 'Open', 'High', 'Low', 'Close']]

In [7]:
lsrg_df = moex_df[(moex_df['BOARDID'] == 'TQBR') & (moex_df['SECID'] == 'LSRG')][['SHORTNAME', 'SECID', 'BOARDID', 'NUMTRADES', 'Value', 'Volume', 'Open', 'High', 'Low', 'Close']]

In [8]:
sibn_df.head()

,SHORTNAME,SECID,BOARDID,NUMTRADES,Value,Volume,Open,High,Low,Close
TRADEDATE,,,,,,,,,,
2015-01-05,Газпрнефть,SIBN,TQBR,180,2289392.0,16230,140.2,145.9,139.3,142.0
2015-01-06,Газпрнефть,SIBN,TQBR,198,3584543.0,24970,142.3,144.6,142.2,144.6
2015-01-08,Газпрнефть,SIBN,TQBR,478,9128362.0,61680,144.5,149.5,144.5,148.1
2015-01-09,Газпрнефть,SIBN,TQBR,464,4965042.0,33980,147.4,148.4,143.3,146.6
2015-01-12,Газпрнефть,SIBN,TQBR,307,3866865.0,26650,144.6,147.7,143.0,143.1


In [9]:
gazp_df.head()

,SHORTNAME,SECID,BOARDID,NUMTRADES,Value,Volume,Open,High,Low,Close
TRADEDATE,,,,,,,,,,
2015-01-05,ГАЗПРОМ ао,GAZP,TQBR,27262,2.410731e+09,18223370,129.50,133.95,129.15,133.95
2015-01-06,ГАЗПРОМ ао,GAZP,TQBR,30998,3.451542e+09,25297090,133.00,138.92,132.71,138.92
2015-01-08,ГАЗПРОМ ао,GAZP,TQBR,46904,4.739749e+09,32604900,140.05,147.50,140.00,146.46
2015-01-09,ГАЗПРОМ ао,GAZP,TQBR,44720,4.203773e+09,29508430,146.48,147.08,139.90,141.70
2015-01-12,ГАЗПРОМ ао,GAZP,TQBR,37074,3.577341e+09,25292180,141.34,143.44,139.72,140.22


In [10]:
lsrg_df.head()

,SHORTNAME,SECID,BOARDID,NUMTRADES,Value,Volume,Open,High,Low,Close
TRADEDATE,,,,,,,,,,
2015-01-06,ЛСР ао,LSRG,TQBR,102,380524.0,805,472.0,485.0,467.0,473.0
2015-01-08,ЛСР ао,LSRG,TQBR,130,727820.0,1505,478.0,495.0,476.0,480.0
2015-01-09,ЛСР ао,LSRG,TQBR,174,584293.0,1212,484.0,490.0,465.0,489.0
2015-01-12,ЛСР ао,LSRG,TQBR,218,626999.0,1311,487.0,488.0,463.0,484.0
2015-01-13,ЛСР ао,LSRG,TQBR,144,944520.0,2015,482.0,485.0,465.0,468.0


In [11]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 30

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

In [12]:
sibn_bt = Backtest(sibn_df, SmaCross, cash=10000, commission=.002)

In [13]:
sibn_bt.run()

Start                     2015-01-05 00:00:00
End                       2019-09-27 00:00:00
Duration                   1726 days 00:00:00
Exposure [%]                          95.7126
Equity Final [$]                      20668.2
Equity Peak [$]                       23048.9
Return [%]                            106.682
Buy & Hold Return [%]                 197.887
Max. Drawdown [%]                    -27.1469
Avg. Drawdown [%]                     -2.6115
Max. Drawdown Duration      341 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                   31
Win Rate [%]                          45.1613
Best Trade [%]                        50.1113
Worst Trade [%]                        -10.86
Avg. Trade [%]                        2.84638
Max. Trade Duration         377 days 00:00:00
Avg. Trade Duration          54 days 00:00:00
Expectancy [%]                         6.9728
SQN                                   1.19847
Sharpe Ratio                      

In [14]:
sibn_bt.plot()

In [15]:
gazp_bt = Backtest(gazp_df, SmaCross, cash=10000, commission=.002)

In [16]:
gazp_bt.run()

Start                     2015-01-05 00:00:00
End                       2019-09-27 00:00:00
Duration                   1726 days 00:00:00
Exposure [%]                          95.9444
Equity Final [$]                      4975.58
Equity Peak [$]                       10606.9
Return [%]                           -50.2442
Buy & Hold Return [%]                 70.9668
Max. Drawdown [%]                    -64.0704
Avg. Drawdown [%]                    -1.05405
Max. Drawdown Duration        3 days 00:00:00
Avg. Drawdown Duration        3 days 00:00:00
# Trades                                   48
Win Rate [%]                            31.25
Best Trade [%]                        34.8919
Worst Trade [%]                       -10.561
Avg. Trade [%]                       -1.25577
Max. Trade Duration         112 days 00:00:00
Avg. Trade Duration          35 days 00:00:00
Expectancy [%]                        4.85247
SQN                                  -1.85845
Sharpe Ratio                      

In [17]:
gazp_bt.plot()

In [18]:
lsrg_bt = Backtest(lsrg_df, SmaCross, cash=10000, commission=.002)

In [19]:
lsrg_bt.run()

Start                     2015-01-06 00:00:00
End                       2019-09-27 00:00:00
Duration                   1725 days 00:00:00
Exposure [%]                          92.8696
Equity Final [$]                      5169.77
Equity Peak [$]                         10000
Return [%]                           -48.3023
Buy & Hold Return [%]                 54.5455
Max. Drawdown [%]                    -56.3439
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                   59
Win Rate [%]                          33.8983
Best Trade [%]                        23.3589
Worst Trade [%]                      -14.9425
Avg. Trade [%]                      -0.901128
Max. Trade Duration          95 days 00:00:00
Avg. Trade Duration          28 days 00:00:00
Expectancy [%]                         4.5185
SQN                                  -1.41755
Sharpe Ratio                      

In [20]:
lsrg_bt.plot()